In [1]:
from pdfparser import Parser
from pdfparser import show_page
import pdfplumber
from pdfplumber.page import Page
from pdfplumber.table import Table
from typing import List, Dict


In [2]:
parser=Parser('test.PDF')

Msg from find_header: 页眉为'2021年年度报告'
Msg from find_toc: 在第4页找到目录


In [ ]:
parser.consolidate_tables()

In [10]:
class TableMsg:
    def __init__(self,title:str,page_num:int) -> None:
        self.title=title
        self.table:Table=None
        self.unit_text:str =None
        self.page_num=page_num
        self.whether_applies=False
    def __str__(self) -> str:
        s=''
        if self.whether_applies:
            s=self.title
        return s
    def __repr__(self) -> str:
        s=''
        if self.whether_applies:
            s='适用'
        else:
            s='不适用'
        return "%s(%s)  在第%d页"%(self.title,s,self.page_num)

# 关于坐标位置的说明
    关于坐标位置的说明
    bbox记录四个位置信息(x0, top, x1, bottom)用来标记一个矩形区域,
    下面是项目地址(https://github.com/jsvine/pdfplumber)中的部分文档说明
    x0	Distance of left-side extremity from left side of page.
    x1	Distance of right-side extremity from left side of page.
    y0	Distance of bottom extremity from bottom of page.
    y1	Distance of top extremity bottom of page.
    top	Distance of top of line from top of page.
    bottom	Distance of bottom of the line from top of page.
    doctop	Distance of top of line from top of document.

In [11]:
def rerange_words(words: List[Dict[str, str]], position: str = 'top'):#根据坐标信息重新排列字词块
    index_dict:Dict[float,int]={}
    rerange_words_list: List[List[Dict[str, str]]] = []
    if position not in {'x0', 'x1', 'top', 'doctop', 'bottom'} or len(words) == 0:
        return rerange_words_list,index_dict
    pre_word_position = None
    index=0
    for i in range(1, len(words)):
        word = words[i]
        if word[position] != pre_word_position:
            index_dict[word[position]]=index
            index+=1
            rerange_words_list.append([word])
            pre_word_position = word[position]
        else:
            rerange_words_list[-1].append(word)
    return rerange_words_list,index_dict

def find_words_in_pdf(self:Parser,target_str_or_list=None):#words_str_or_list="√不适用,√适用"
    def match_words(match_words,target_words):
        tmp_page_words = []
        for match in match_words:
            for word in target_words:
                if word == match['text']:
                    tmp_page_words.append(match)
        return tmp_page_words
    words_msg: List[List[Dict[str]]] = []  # 列表中的每一个元素存储一个页的目标字词块信息
    target_words = target_str_or_list
    if isinstance(target_str_or_list, str):
        target_words = target_str_or_list.split(',')
    for matchs in self.all_page_words:
        tmp_page_words=match_words(match_words=matchs,target_words=target_words)
        words_msg.append(tmp_page_words)
    return words_msg

'''
先拿一个页面作为例子，新找到页面中的所有是否适用信息
重新按行排列字词块
再通过适用信息的位置找到其上方的字词块
'''
def creat_table_msg(self:Parser):
    def text_on_whether_applies_msg(target_words:List[Dict[str]],words:List[Dict[str, str]]):
        title_list=[]
        whether_applies_list=[]
        title='未找到标题'
        whether_applies=False
        range_words,index_dict=rerange_words(words)
        for target in target_words:
            range_words_index=index_dict[target['top']]
            
            if target['text']=='√适用':
                whether_applies=True
                
            if range_words_index<0:
                #标题可能在上一页
                pass
            else:
                title=range_words[range_words_index-1][-1]['text']
            title_list.append(title)
            whether_applies_list.append(whether_applies)
        return title_list,whether_applies_list
    
    table_msg_list:List[TableMsg]=[]
    target_words_list=find_words_in_pdf(self,"√不适用,√适用")
    for index,target_words in enumerate(target_words_list):
            if len(target_words)==0:
                continue
            title_list,whether_applies_list=text_on_whether_applies_msg(target_words,self.new_page_words[index])
            for i in range(len(title_list)):
                table_msg=TableMsg(title=title_list[i],page_num=self.cropped_pages[index].page_number)
                table_msg.whether_applies=whether_applies_list[i]
                table_msg_list.append(table_msg)
    return table_msg_list
creat_table_msg(parser)

[是否存在公司治理特殊安排等重要事项(不适用)  在第2页,
 前瞻性陈述的风险声明(适用)  在第3页,
 其他(适用)  在第3页,
 公司股票简况(适用)  在第7页,
 公司存托凭证简况(适用)  在第7页,
 报告期末公司前三年主要会计数据和财务指标的说明(适用)  在第8页,
 的净资产差异情况(不适用)  在第9页,
 净资产差异情况(不适用)  在第9页,
 境内外会计准则差异的说明：(不适用)  在第9页,
 季度数据与已披露定期报告数据差异说明(不适用)  在第9页,
 九、非经常性损益项目和金额(适用)  在第9页,
 项目界定为经常性损益项目的情况说明(不适用)  在第10页,
 十、采用公允价值计量的项目(适用)  在第10页,
 十一、非企业会计准则业绩指标说明(适用)  在第10页,
 国家科学技术奖项获奖情况(不适用)  在第18页,
 国家级专精特新“小巨人”企业、制造业“单项冠军”认定情况(不适用)  在第18页,
 研发投入总额较上年发生重大变化的原因(不适用)  在第19页,
 研发投入资本化的比重大幅变动的原因及其合理性说明(适用)  在第19页,
 在研项目情况(适用)  在第20页,
 研发人员构成发生重大变化的原因及对公司未来发展的影响(不适用)  在第23页,
 其他说明(不适用)  在第23页,
 核心竞争力分析(适用)  在第23页,
 报告期内发生的导致公司核心竞争力受到严重影响的事件、影响分析及应对措施(适用)  在第25页,
 尚未盈利的风险(不适用)  在第26页,
 业绩大幅下滑或亏损的风险(适用)  在第26页,
 核心竞争力风险(适用)  在第26页,
 经营风险(适用)  在第27页,
 财务风险(不适用)  在第28页,
 行业风险(适用)  在第28页,
 宏观环境风险(适用)  在第28页,
 存托凭证相关风险(适用)  在第28页,
 其他重大风险(适用)  在第28页,
 本期公司业务类型、利润构成或利润来源发生重大变动的详细说明(不适用)  在第29页,
 收入和成本分析(适用)  在第30页,
 产销量情况分析表(适用)  在第31页,
 重大采购合同、重大销售合同的履行情况(适用)  在第31页,
 报告期主要子公司股权变动导致合并范围变化(不适用)  在第32页,
 公司报告期内

In [6]:
def find_unit(self:Parser):#按垂直顺序对字块排序
    page=self.cropped_pages[29]
    words=self.all_page_words[29]
    tables=page.find_tables()
    def find_pre_table_words(table:Table,words:List[Dict[str, str]]):
        words_num=len(words)
        top_on_table_words:List[Dict[str, str]]=[]
        index=0
        while index<words_num:
            word=words[index]
            tmp_words:List[Dict[str, str]]=[]
            if index==words_num:
                print('Error from find_unit.top_table_word: 未找到指定字符块，理论上不存在这种情况，请检查该方法存在的问题。')
                return False
            if word['top']<table.bbox['top']:
                pass
            index+=1
    unit_words=rerange_words(words=words)
    for word in unit_words:
        print(word)
    
find_unit(parser)

[[{'text': '2.', 'x0': 89.904, 'x1': 100.464, 'top': 77.68895999999995, 'doctop': 23753.568959999986, 'bottom': 88.24896000000001, 'upright': True, 'direction': 1}, {'text': '收入和成本分析', 'x0': 110.9, 'x1': 184.82000000000002, 'top': 77.68895999999995, 'doctop': 23753.568959999986, 'bottom': 88.24896000000001, 'upright': True, 'direction': 1}], [{'text': '√适用', 'x0': 89.904, 'x1': 121.58, 'top': 94.38895999999988, 'doctop': 23770.268959999987, 'bottom': 104.94895999999994, 'upright': True, 'direction': 1}, {'text': '□不适用', 'x0': 126.74, 'x1': 168.74384, 'top': 94.38895999999988, 'doctop': 23770.268959999987, 'bottom': 104.94895999999994, 'upright': True, 'direction': 1}], [{'text': '报告期内，公司实现营业收入68,468.39万元，较上年同期减少40,343.31万元，同比减少', 'x0': 110.9, 'x1': 531.3399999999999, 'top': 107.94895999999994, 'doctop': 23783.828959999988, 'bottom': 118.50896, 'upright': True, 'direction': 1}], [{'text': '37.08%，主要系受疫情持续的影响，一类新药帕拉米韦氯化钠注射液销量减少。公司发生营业成', 'x0': 89.904, 'x1': 531.3515199999999, 'top': 128.